In [2]:
a = {1:2}

In [21]:
print(a[1])

2


In [ ]:
class LRUCache(collections.OrderedDict):
    
    def __init__(self, capacity: int):
        super().__init__()
        self.capacity = capacity
        
    def get(self, key: int) -> int:
        if key not in self:
            return -1
        self.move_to_end(key)
        return self[key]

    def put(self, key: int, value: int) -> None:
        if key in self:
            self.move_to_end(key)
        self[key] = value
        if len(self) > self.capacity:
            self.popitem(last=False)

In [22]:
# O(n) 的时间复杂度
from collections import defaultdict

class LRUCache:
    def __init__(self, capacity: int):
        self.idx = 0
        self.capacity = capacity
        self.cache = {}
        self.freq = []

    def get(self, key: int) -> int:
        if key in self.cache:
            self.freq.remove(key)
            self.freq.append(key)
            return self.cache[key]
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            self.freq.remove(key)
        else:
            if self.idx + 1 > self.capacity:
                del_key = self.freq.pop(0)
                del self.cache[del_key]
            else:
                self.idx += 1
        self.cache[key] = value
        self.freq.append(key)

<img src='146.jpg'> <img src='146_2.jpg'>

In [ ]:
# 1、使用双向链表，可以实现时间复杂度 为 O(1) 的 put 和 get
# 每次 get 的时候，需要把 get 的节点移动至双向链表的头步，每次如果超过capicity
# 删除的都是链表的尾节点

class DLinkedNode:
    def __init__(self):
        self.key = 0
        self.val = 0
        self.next = None
        self.prev = None

class LRUCache:
    def __init__(self, capacity: int):
        self.cache = {}
        self.capacity = capacity 
        self.size = 0
        # 构建双向链表
        self.head, self.tail = DLinkedNode(), DLinkedNode()
        self.head.next = self.tail
        self.tail.prev = self.head

    def get(self, key: int) -> int:
        node = self.cache.get(key, None)
        if not node:
            return -1
        self._move_to_head(node) # 将访问的节点移动到双向链表的头节点
        return node.val

    def put(self, key: int, value: int) -> None:
        node = self.cache.get(key)
        if not node: # 当前节点不在 cache 中
            newNode = DLinkedNode()
            newNode.key = key
            newNode.val = value
            
            self.cache[key] = newNode # cache中存储的是 [key, node]
            self._add_node(newNode)
            self.size += 1

            if self.size > self.capacity:
                tail = self._pop_tail() # 大于capacity, 删除的是尾结点
                del self.cache[tail.key]
                self.size -= 1
        else:
            node.val = value
            self._move_to_head(node) # 更新存在的节点，移动至头部

    def _add_node(self, node): # 添加 node 节点到首节点的后面
        node.prev = self.head
        node.next = self.head.next
        
        self.head.next.prev = node
        self.head.next = node

    def _remove_node(self, node): # 删除 node 节点
        prev = node.prev
        nex = node.next

        prev.next = nex
        nex.prev = prev
        
    def _move_to_head(self, node): # 移动 node 节点到head
        self._remove_node(node) # 先删除当前这个节点，再把当前节点加入到头节点之后
        self._add_node(node)

    def _pop_tail(self): # 删除尾部的 node 节点
        tail = self.tail.prev # 获得尾部节点 prev 节点
        self._remove_node(tail) # 删除该节点
        return tail

# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [23]:
# 1、使用双向链表，可以实现时间复杂度 为 O(1) 的 put 和 get
# 每次 get 的时候，需要把 get 的节点移动至双向链表的头步，每次如果超过capicity
# 删除的都是链表的尾节点

class DLinkedNode:
    def __init__(self):
        self.key = 0 # 如果不存储 key，则无法直到删除的尾结点是什么,没有必要存储 value
        self.next = None
        self.prev = None

class LRUCache:
    def __init__(self, capacity: int):
        self.cache = {}
        self.capacity = capacity 
        self.size = 0
        # 构建双向链表
        self.head, self.tail = DLinkedNode(), DLinkedNode()
        self.head.next = self.tail
        self.tail.prev = self.head

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        val, node = self.cache[key]
        self._move_to_head(node) # 将访问的节点移动到双向链表的头节点
        return val

    def put(self, key: int, value: int) -> None:
        if key not in self.cache: # 当前节点不在 cache 中
            newNode = DLinkedNode()
            newNode.key = key
            
            self.cache[key] = [value, newNode] # cache中存储的是 [key:, [val, node]]
            self._add_node(newNode)
            self.size += 1

            if self.size > self.capacity:
                tail = self._pop_tail() # 大于capacity, 删除的是尾结点
                del self.cache[tail.key]
                self.size -= 1
        else:
            _, node = self.cache[key]
            self.cache[key][0] = value
            self._move_to_head(node) # 更新存在的节点，移动至头部

    def _add_node(self, node): # 添加 node 节点到首节点的后面
        node.prev = self.head
        node.next = self.head.next
        
        self.head.next.prev = node
        self.head.next = node

    def _remove_node(self, node): # 删除 node 节点
        prev = node.prev
        nex = node.next

        prev.next = nex
        nex.prev = prev
        
    def _move_to_head(self, node): # 移动 node 节点到head
        self._remove_node(node) # 先删除当前这个节点，再把当前节点加入到头节点之后
        self._add_node(node)

    def _pop_tail(self): # 删除尾部的 node 节点
        tail = self.tail.prev # 获得尾部节点 prev 节点
        self._remove_node(tail) # 删除该节点
        return tail

# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [24]:
lRUCache = LRUCache(2)
lRUCache.put(1, 1); # 缓存是 {1=1}
lRUCache.put(2, 2); # 缓存是 {1=1, 2=2}
lRUCache.get(1);    # 返回 1
lRUCache.put(3, 3); # 该操作会使得关键字 2 作废，缓存是 {1=1, 3=3}
lRUCache.get(2);    # 返回 -1 (未找到)
lRUCache.put(4, 4); # 该操作会使得关键字 1 作废，缓存是 {4=4, 3=3}
lRUCache.get(1);    # 返回 -1 (未找到)
lRUCache.get(3);    # 返回 3
lRUCache.get(4);    # 返回 4